In [ ]:
import json
import pandas

!smt export
with open('.smt/records_export.json') as ff:
    data = json.load(ff)
    
with open('record1.json', 'w') as record1:
    for entry in range(len(data)):
        record1.write(json.dumps(data[entry], sort_keys=True, indent=4, separators=(',', ': ')))

df = pandas.DataFrame(data)

In [ ]:
fn = np.load('/data/aem1/new1a/surf-research/Andrey/Data/1de38ff12ec7/1a10x1step20.npz')
fn['time']
# del results
# print results

In [ ]:
import os
import numpy as np

results = {}

#This code will access every step file saved in the Data/[label] directory 
for label in df['label']:
    filepath = os.path.join('/data/aem1/new1a/surf-research/Andrey/Data', label)
    #gets the last step file in the directory (hopefully at equillibrium)
    file = os.listdir('Data/{0}'.format(label))[len(os.listdir('Data/{0}'.format(label)))-2] 
    if file == 'memory_usage.txt':
        break #the memory file should always be the last one saved
    else:
        fn = np.load(os.path.join(filepath, file))
        print file
        size = file[2 : file.find('s')] #the grid size based on filename
        appender = {}
        #for every simulation time, create a dictionary entry to match the cvar values at that time
        for x in xrange(len(fn['time'])):
            appender.update({fn['time'][x]:fn['c_var'][x]})
            
        print fn["time"]

        #add a dictionary entry to associate the size of the grids with the dictionaries of times and their cvar outputs
        results.update({size:appender})
        
print results['14x1']
#now we can load this dictionary into a dataframe or interpolate

In [ ]:
fn['time']

In [ ]:
print len(df['output_data'][0])

In [ ]:
for p in 'N', 'steps', 'sweeps':
    df[p] = df.parameters.map(lambda x: json.loads(x['content'])[p])

In [ ]:
print df['parameters'][0]['content']

In [ ]:
df

In [ ]:
columns = ['label', 'N', 'steps', 'tags', 'duration', 'sweeps']
print df[columns]

In [ ]:
import numpy as np
import os
num = 0
memoryArray = np.array([])
for num in range(len(df.index)):
    memName = df['output_data'][num][len(df['output_data'][num])-1]['path']
    memlocation = df['datastore'][num]['parameters']['root']
    pool = os.path.join(memlocation, memName)
    memdata = np.loadtxt(pool)
    memoryArray = np.append(memoryArray, memdata)
    print memoryArray
    print ' '
    print num

In [ ]:
df['memory']=memoryArray
columns.append('memory')
df[columns]